# Kaleabe Negussie UGR/3411/12

In [50]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import re
from tqdm import  tqdm
import os
import collections
import gc
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist, FreqDist

In [52]:
file_path = "/kaggle/input/amharic-corpus-general/GPAC.txt"
n = 1

In [53]:
os.path.exists(file_path)
if os.path.exists(file_path):
    print(os.stat(file_path).st_size / (1024 * 1024))
else:
    print(os.path.exists(file_path))

1013.849326133728


# 1) N-gram language model

## 1.1 Create n-grams for n=1, 2, 3, 4. You can show sample prints.

In [54]:
# Downloading necessary NLTK data
nltk.download('punkt')

def process_chunk(chunk, n):
    """
    Process a chunk of text to create n-grams of size n.
    """
    tokens = word_tokenize(re.sub(r'\W+', ' ', chunk))
    return list(ngrams(tokens, n))

def read_and_process_in_chunks(file_path, n, chunk_size=1024*1024):
    """
    Read the file in chunks and process each chunk for n-grams of size n.
    """
    with open(file_path, 'r') as file:  # Changed to 'latin1'
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            yield process_chunk(chunk, n)

# Usage example
# file_path = '/content/GPAC.txt'  # Adjust this path to your uploaded file
# n = 2  # Example for bigrams; change as needed

all_ngrams = []
for ngrams_chunk in read_and_process_in_chunks(file_path, n):
    all_ngrams.extend(ngrams_chunk)
    gc.collect()

# Optionally, print some n-grams to check
# print(all_ngrams[:10])  # Displaying the first 10 n-grams

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


In [56]:
print(len(all_ngrams))

77760256


In [58]:
print(all_ngrams[:10])

[('ምን',), ('መሰላችሁ',), ('አንባቢያን',), ('ኢትዮጵያ',), ('በተደጋጋሚ',), ('ጥሪው',), ('ደርሷት',), ('ልትታደመው',), ('ያልቻለችው',), ('የአለም',)]


## 1.2 Calculate probabilities of n-grams and find the top 10 most likely n-grams for all n.

In [59]:
# Create a conditional frequency distribution of Amharic n-grams
def create_freq_dist(ngrams):
    cfreq = ConditionalFreqDist((tuple(ngram[:-1]), ngram[-1]) for ngram in ngrams)
    return cfreq

# Create a conditional probability distribution using maximum likelihood estimation
def create_prob_dist(cfreq):
    cprob = ConditionalProbDist(cfreq, MLEProbDist)
    return cprob


probabilites = []

cfreq = create_freq_dist(all_ngrams)
cprob = create_prob_dist(cfreq)
probabilities = cprob
gc.collect()

    

0

In [60]:
# Get the top n-grams with the highest probabilities
def get_top_ngrams(cfreq, n, top_k):
    top_ngrams = []
    for context in cfreq.conditions():
        if len(context) == n - 1:
            freq_dist = cfreq[context]
            top_k_ngrams = freq_dist.most_common(top_k)
            top_ngrams.extend([(context + (ngram,), freq_dist.freq(ngram)) for ngram in top_k_ngrams])
    return top_ngrams

In [62]:
top_ngrams = get_top_ngrams(cfreq, n, 10)
for ngram, probability in top_ngrams:
    print(ngram)

(('ነው', 1166198),)
(('ላይ', 754597),)
(('ውስጥ', 346288),)
(('ወደ', 330973),)
(('እና', 318782),)
(('ጋር', 291520),)
(('ነበር', 275023),)
(('ግን', 271505),)
(('ጊዜ', 244085),)
(('ነገር', 217662),)


## 1.3 What is the probability of the sentence. "ኢትዮጵያ ታሪካዊ ሀገር ናት ". You can also try more sentences.

In [ ]:
sentence = ("ኢትዮጵያ" ,"ታሪካዊ" ,"ሀገር", "ናት")
# Calculate the probability of a specific four-gram
def calculate_fourgram_probability(cprob, fourgram):
    context = tuple(fourgram[:-1])
    word = fourgram[-1]
    probability = cprob[context].prob(word)
    return probability

probability = calculate_fourgram_probability(probabilities[3], sentence)

## 1.4 Generate random sentences using n-grams; explain what happens as n increases, based on your output.

In [63]:
# Generate a sentence using unigrams
def generate_sentence(cpd, max_length=15):
    sentence = []
    word = None
    while len(sentence) < max_length:
        try:
            word = cpd[()].generate() if word is None else cpd[(word,)].generate()
        except IndexError:
            break  # Stop generating if context is not found in the conditional distribution
        if word == '።':
            break
        sentence.append(word)
    return ' '.join(sentence)

In [64]:
# Generate a sentence using the n-gram model
generated_sentence = generate_sentence(probabilities)
print(generated_sentence)

ወንጀል
